# colab을 이용한 Natural Language Processing(NLP) 실습

🎯 학습 목표 : colab 환경에서 NLP 모델 학습 코드를 실행하고 결과를 확인할 수 있다.

- 실습 재료

| 항목 | 상세 |
| ---- | ---- |
| 🗂️ 데이터 | AI-HUB의 감성 대화 말뭉치 중 샘플 데이터 |
| 🤖 NLP 언어 모델 | GPT-2 (Generative pre-trained transformer-2) |
| 🏗️ NLP 학습 프레임워크 | torch |
| 🐍 프로그래밍 언어 | Python |
| 👩‍💻 프로그래밍 환경 | Colab |


- colab에서 코드 실행 방법은 다음 그림을 참조해주시기 바랍니다.
    ![](https://i.imgur.com/0GoFr7q.png)

### 1. 데이터 로딩 및 전처리

이번 실습에서 사용되는 데이터는 AI-HUB의 치매 감성 대화 말뭉치 중 샘플 데이터 집합 중 인지기능 데이터의 샘플 자료를 이용합니다.

실습 데이터 집합에 대한 명세는 다음과 같습니다.

- 데이터 개수 : 19,920개
- 주요 항목
  - persona: 대화자의 페르소나 정보
  - emotion: 감정 정보(우리가 예측하고자 하는 타겟)
  - talk: 대화 내용
    - content: 대화 내용 (HS01, SS01, ...)

우리가 원하는 것은 각 대화(HS01, SS01, ...)의 내용과 그에 해당하는 감정(emotion의 type)입니다. 따라서 이 정보를 추출하여 전처리를 진행하겠습니다.

In [ ]:
!git clone https://github.com/ShinJM-maker/pratice_GPT2.git

In [ ]:
import json

# 로컬에서 JSON 파일 읽기
file_path = "/content/pratice_GPT2/감성대화말뭉치(최종데이터)_Validation.json"  # 실제 파일 경로로 변경해주세요
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Extracting relevant data for emotion classification
texts = []
labels = []

for entry in data:
    emotion = entry['profile']['emotion']['type']

    # Extracting all the dialogues
    for key, dialogue in entry['talk']['content'].items():
        if key.startswith('HS'):  # We'll only consider human dialogues for simplicity
            texts.append(dialogue)
            labels.append(emotion)

# Checking the first few entries
texts[100:105], labels[100:105]


위 코드의 실행으로 데이터가 성공적으로 추출되었습니다. 각 대화 내용(texts)과 해당 대화의 감정 라벨(labels)을 확인할 수 있습니다. 데이터를 학습용과 검증용으로 나누겠습니다.

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

len(train_texts), len(val_texts)


데이터를 학습용(15,936개)과 검증용(3,984개)으로 나눴습니다.

아래는 AI-HUB의 감성 대화 말뭉치의 라벨 목록입니다.

![Image Alt Text](https://drive.google.com/uc?export=view&id=1fzxQIpz21nOc8G1nP8AFuERijwoUodaV)


아래는 데이터셋의 라벨별 개수를 시각화 하는 코드입니다.

In [ ]:
from collections import Counter

# Count the occurrences of each label in the training set
label_distribution = Counter(train_labels)
#print(label_distribution)
sorted_label_distribution = dict(sorted(label_distribution.items(), key=lambda item: int(item[0][1:])))
print(sorted_label_distribution)

import matplotlib.pyplot as plt

# Extract labels and their counts
labels = list(sorted_label_distribution.keys())
counts = list(sorted_label_distribution.values())

# Plot
plt.figure(figsize=(15, 7))  # Set the figure size for better visibility
plt.bar(labels, counts, color='blue')
plt.xlabel('Emotion Labels')
plt.ylabel('Number of Occurrences')
plt.title('Distribution of Emotion Labels')
plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility
plt.tight_layout()  # Adjust layout for better visibility
plt.show()


### 2. GPT-2 모델 및 토크나이저 로딩

다음 단계로, GPT-2 모델 및 토크나이저를 로딩하겠습니다. Google Colab에서 실행할 때, 라이브러리 설치 및 모델 로딩을 위한 코드를 제공하겠습니다.

본 과정에서는 다음을 수행합니다.

1. 필요한 라이브러리 설치
2. GPT-2 토크나이저 로딩
3. 라벨 인코딩 및 텍스트 토큰화
4. PyTorch 데이터셋으로 변환
5. GPT-2 모델 로딩 및 분류 헤드 추가
6. 학습 설정



아래의 코드에서 필요한 라이브러리를 설치합니다.
1. accelerate
2. transformers

In [ ]:
!pip uninstall accelerate -y
!pip install accelerate

!pip install transformers[torch] -U

라이브러리를 설치했다면 이제 토크나이저를 로딩하고 학습을 위한 설정을 진행합니다.

아래 코드를 통해 감정 분류 작업을 위해 GPT-2 모델을 미세 조정하는 방법을 배웁니다.
우리는 GPT-2 모델을 활용하고 분류 작업을 위해 시퀀스 분류 헤드를 추가함으로써 이를 적용시킬 것입니다.

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import DataLoader

# 토큰화: GPT-2 토큰화기를 로드하여 시작합니다.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

#라벨 인코딩: 분류 작업을 위해 문자열 라벨을 정수로 변환해야 합니다.

combined_labels = train_labels + val_labels

label_encoder = LabelEncoder()
label_encoder.fit(combined_labels)

train_enc_labels = label_encoder.transform(train_labels)
val_enc_labels = label_encoder.transform(val_labels)

# 텍스트 인코딩: GPT-2 토큰화기를 사용하여 텍스트 데이터를 모델이 이해할 수 있는 형식으로 변환합니다.
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=100)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=100)

# PyTorch 데이터셋 생성: 데이터가 토큰화되면 이를 PyTorch Dataset으로 변환해야 합니다.
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmotionDataset(train_encodings, train_enc_labels)
val_dataset = EmotionDataset(val_encodings, val_enc_labels)

# 모델 초기화: 시퀀스 분류 헤드가 있는 사전 훈련된 GPT-2 모델을 로드합니다.
model = GPT2ForSequenceClassification.from_pretrained("gpt2-medium", num_labels=len(set(combined_labels)))

# 모델 구성: 토큰화기에 패딩 토큰을 설정한 것처럼 모델에게도 이 패딩 토큰에 대해 알려야 합니다.
model.config.pad_token_id = model.config.eos_token_id

model = model.to("cuda" if torch.cuda.is_available() else "cpu")

# 훈련 설정: 우리는 오류를 최소화하기 위해 훈련 중에 모델의 가중치를 조정하는 최적화기를 정의합니다.
optimizer = AdamW(model.parameters(), lr=2e-5)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


### 3. GPT-2를 활용한 감정 분류 모델 훈련 및 평가

이 섹션에서는 GPT-2 감정 분류 모델을 훈련하고 평가하는 과정을 다룹니다. 우리는 훈련 데이터를 활용하여 모델을 미세 조정하고, 검증 데이터를 사용하여 모델의 성능을 평가합니다.

본 과정에서는 다음을 수행합니다.

1. 훈련 모드 설정
2. 훈련 루프
3. 평가 모드 설정
4. 모델 평가
5. 모델 및 토큰화기 저장

원래는 여러 학습 epoch를 진행해야 하지만, 실습을 간략하게 진행하기 위해 1 epoch와 10 batch만으로 학습을 수행합니다.

In [ ]:
from sklearn.metrics import f1_score

# 보다 빠른 학습을 위해 1 epoch만 실행하는 학습 루프
for epoch in range(1):
    model.train()  # 모델을 학습 모드로 설정
    total_train_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()  # 그라디언트 초기화
        # 입력 데이터 GPU 사용 가능 여부에 따라 설정
        inputs = {k: v.to("cuda" if torch.cuda.is_available() else "cpu") for k, v in batch.items()}
        outputs = model(**inputs)  # 모델에 입력 데이터 전달
        loss = outputs.loss  # 손실 값
        total_train_loss += loss.item()  # 총 학습 손실에 추가
        loss.backward()  # 역전파
        optimizer.step()  # 최적화 수행

        print(f"Batch {batch_idx}, Training Loss: {total_train_loss / (batch_idx + 1)}")
        if batch_idx == 10:
            break

    # 모델 평가
    model.eval()  # 모델을 평가 모드로 설정
    total_val_loss = 0
    all_predictions = []
    all_labels = []
    batch_count = 0  # 배치 수 제한용 변수
    with torch.no_grad():  # 그라디언트 계산 비활성화
        for batch in val_loader:
            batch_count += 1
            # 디버깅을 위해 처음 5개 배치만 처리
            if batch_count > 5:
                break

            print(f"Processing batch {batch_count}...")  # 진행 상황 모니터링
            inputs = {k: v.to("cuda" if torch.cuda.is_available() else "cpu") for k, v in batch.items()}
            labels = batch['labels'].to("cuda" if torch.cuda.is_available() else "cpu")
            outputs = model(**inputs)
            total_val_loss += outputs.loss.item()
    print(f"Epoch {epoch + 1}, Validation Loss: {total_val_loss / 5}")

# 모델 및 토크나이저 저장
model.save_pretrained("/content/pratice_GPT2/gpt2_emotion_classifier")
tokenizer.save_pretrained("/content/pratice_GPT2/gpt2_emotion_classifier")

### 4. 학습된 모델 Inference(실행)

아래는 학습된 모델을 실제 실행해볼 수 있는 코드입니다.

본 과정에서는 다음을 수행합니다.

1. 데이터셋 라벨 사전화
2. 학습된 모델을 Inferece하는 함수 작성
3. 텍스트 입력을 통한 모델 Inference

우선 주어진 데이터셋의 라벨을 사전(Dictionary)화 하여, Index(키 값)이 들어오면 한글로 반환하게 합니다.

사전이 필요한 이유는 모델의 출력이 텍스트가 아닌 60개 감정의 index(숫자) 이기 때문에 이를 텍스트로 매핑시켜 출력하는 과정이 필요합니다.

In [ ]:
# 주어진 텍스트 데이터
text_data = [
    ["분노", "슬픔", "불안", "상처", "당황", "기쁨"],
    ["툴툴대는", "실망한", "두려운", "질투하는", "고립된", "감사하는"],
    ["좌절한", "비통한", "스트레스 받는", "배신당한", "남의 시선 의식하는", "사랑하는"],
    ["짜증나는", "후회되는", "취약한", "고립된", "외로운", "편안한"],
    ["방어적인", "우울한", "혼란스러운", "충격 받은", "열등감", "만족스러운"],
    ["악의적인", "마비된", "당혹스러운", "불우한", "죄책감", "흥분되는"],
    ["안달하는", "염세적인", "회의적인", "희생된", "부끄러운", "느긋한"],
    ["구역질 나는", "눈물이 나는", "걱정스러운", "억울한", "혐오스러운", "안도하는"],
    ["노여워하는", "낙담한", "조심스러운", "괴로워하는", "한심한", "신이 난"],
    ["성가신", "환멸을 느끼는", "초조한", "버려진", "혼란스러운", "자신하는"]
]

# 빈 딕셔너리 생성
result_dict = {}

# 텍스트 데이터를 기반으로 딕셔너리 생성
for i, row in enumerate(text_data):
    for j, emotion in enumerate(row):
        index = i * len(row) + j + 1  # 1부터 시작하는 인덱스
        result_dict[index] = emotion

# 결과 딕셔너리 출력
print(result_dict)

다음은 감정 분류를 위한 GPT-2 모델을 이용하여 주어진 텍스트의 감정을 예측하는 코드입니다.

아래는 텍스트가 주어졌을때 이를 학습된 모델을 통하여 예측하는 함수입니다.

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch

# 모델 및 토크나이저 로드
model_path = "/content/pratice_GPT2/gpt2_emotion_classifier/"
model = GPT2ForSequenceClassification.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def predict_emotion(model, tokenizer, label_encoder, text):

    # 모델을 평가 모드로 설정
    model.eval()

    # 텍스트 토큰화
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=100)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 모델 추론
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class = torch.argmax(logits, dim=1).item()

    return predicted_class

아래의 코드에서 sample_text를 직접 작성하여 감정분류 모델의 출력을 확인해볼 수 있습니다.

In [ ]:
# 예제 텍스트로 감정 예측
sample_text = "친구들에게 큰맘 먹고 속마음을 고백했는데 갑자기 그 이후로 친구들이 나를 따돌렸어."
predicted_emotion = predict_emotion(model, tokenizer, label_encoder, sample_text)
print(f"Predicted Emotion for '{sample_text} \n: {result_dict[predicted_emotion]}")

이상으로 본 실습을 마치도록 하겠습니다.

모두들 고생 많으셨습니다!

![](https://img.favpng.com/10/1/7/kakaotalk-kakao-friends-emoticon-sticker-png-favpng-mZm2vp0mk2Ce9aTUnBjC4s4DZ.jpg)